In [ ]:
import pandas as pd

In [ ]:
from matplotlib import pyplot

In [ ]:
import numpy as np

In [ ]:
import seaborn as sns

In [ ]:
data = pd.read_csv('train.csv')

In [ ]:
data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [ ]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [ ]:
data.corr()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.035144,0.036847,-0.057527,-0.001652,0.012658
Survived,-0.005007,1.000000,-0.338481,-0.077221,-0.035322,0.081629,0.257307
Pclass,-0.035144,-0.338481,1.000000,-0.369226,0.083081,0.018443,-0.549500
Age,0.036847,-0.077221,-0.369226,1.000000,-0.308247,-0.189119,0.096067
SibSp,-0.057527,-0.035322,0.083081,-0.308247,1.000000,0.414838,0.159651
Parch,-0.001652,0.081629,0.018443,-0.189119,0.414838,1.000000,0.216225
Fare,0.012658,0.257307,-0.549500,0.096067,0.159651,0.216225,1.000000


In [ ]:
data.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [ ]:
from collections import Counter

data['Fare']

0       7.2500
1      71.2833
2       7.9250
3      53.1000
4       8.0500
        ...   
886    13.0000
887    30.0000
888    23.4500
889    30.0000
890     7.7500
Name: Fare, Length: 891, dtype: float64

In [ ]:
data_modified = data.drop(columns=['Age', 'Cabin', 'Name', 'PassengerId'])

In [ ]:
data_modified[data_modified['Embarked'].isnull()]

,Survived,Pclass,Sex,SibSp,Parch,Ticket,Fare,Embarked
61,1,1,female,0,0,113572,80.0,NaN
829,1,1,female,0,0,113572,80.0,NaN


In [ ]:
data_modified = data_modified.drop(index=[61,829])

In [ ]:
data_modified = data_modified.drop(columns=['Ticket'])
df_cat = pd.get_dummies(data_modified)
df_cat.columns

Index(['Survived', 'Pclass', 'SibSp', 'Parch', 'Fare', 'Sex_female',
       'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S'],
      dtype='object')

In [ ]:
df_cat_train = df_cat.drop(columns=['Survived'])
from sklearn.preprocessing import Imputer, MinMaxScaler
mms = MinMaxScaler()
normalized_data = mms.fit_transform(df_cat_train)
df_norm = pd.DataFrame(normalized_data, columns=df_cat_train.columns)

In [ ]:

from sklearn.model_selection import train_test_split
X = df_norm.to_numpy()
y = df_cat['Survived']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=15)

In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=12, ratio = 1)
X_train_res, y_train_res = sm.fit_sample(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.feature_extraction import *

def pipelines(X, y, n_feat=9):
  pipelines = [Pipeline([
        ('selector', SelectKBest(chi2, k=n_feat)),
        ('classifier', LogisticRegression(penalty='l2') )
        ])]
  pipelines.append(Pipeline([
      ('selector', SelectKBest(chi2, k=n_feat)),
      ('classifier', MultinomialNB(alpha=1, fit_prior=True))]))
  pipelines.append(Pipeline([
  ('selector', SelectKBest(chi2, k=n_feat)),
      ('classifier', RandomForestClassifier(n_estimators=100, n_jobs=3))]))
  pipelines.append(Pipeline([
      ('selector', SelectKBest(chi2, k=n_feat)),
      ('classifier',  SVC(probability=True, random_state=1, kernel='rbf'))]))
  pipelines.append(Pipeline([
      ('selector', SelectKBest(chi2, k=n_feat)),
      ('classifier',  SVC(probability=True, random_state=1, kernel='linear'))]))
  #pipelines.append(Pipeline([
      #('selector', SelectKBest(chi2, k=n_feat)),
      #('classifier', GradientBoostingClassifier())]))
  #pipelines.append(Pipeline([
      #('selector', SelectKBest(chi2, k=9)),
      #('classifier', KNeighborsClassifier())]))
  return [pip.fit(X, y) for pip in pipelines]


In [ ]:
classifiers = pipelines(X_train_res, y_train_res)
# model validation
for clf in classifiers:
  y_t = clf.predict(X_val)
  print (precision_recall_fscore_support(y_val, y_t))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


(array([0.81578947, 0.71875   ]), array([0.83783784, 0.68656716]), array([0.82666667, 0.70229008]), array([111,  67]))
(array([0.80165289, 0.75438596]), array([0.87387387, 0.64179104]), array([0.8362069 , 0.69354839]), array([111,  67]))
(array([0.79674797, 0.76363636]), array([0.88288288, 0.62686567]), array([0.83760684, 0.68852459]), array([111,  67]))
(array([0.8       , 0.79245283]), array([0.9009009 , 0.62686567]), array([0.84745763, 0.7       ]), array([111,  67]))
(array([0.8       , 0.79245283]), array([0.9009009 , 0.62686567]), array([0.84745763, 0.7       ]), array([111,  67]))


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
scores = ['precision', 'recall']
clf = GridSearchCV(
    SVC(), tuned_parameters, scoring='recall',
)
clf.fit(X_train_res, y_train_res)
print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")



/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Best parameters set found on development set:

{'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}

Grid scores on development set:


In [ ]:
clf.cv_results_

AttributeError: ignored